 <font size="6">GAN — Генеративно-состязательные нейронные сети</font>

# Введение  в генеративно-состязательные нейронные сети

В этом курсе мы, в основном, работали с **размеченными** данными. Мы научили нейронные сети решать задачи классификации, сегментации и т.д. 

В этой лекции мы разберемся, как научить нейросеть создавать что-то новое.


Как подойти к такой задаче с помощью нейронных сетей?

**Постановка задачи генерации**

**Дано**: неразмеченные данные

**Выход**: новые данные, которые будут удовлетворять следующим условиям:
* Новые данные должны быть **похожи** на исходные.
* Но **не повторять** их в точности (или повторять, при случайном стечении обстоятельств).
* Чтобы результат генерации был различным при разных запусках, нам нужен **элемент случайности**.

**Эволюция в генерации изображений лиц:**

[Множество примеров различных генераторов](https://thisxdoesnotexist.com)


<img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/faces_generation_quality_progress.png" width="700">

## Latent space

Разберемся с **элементом случайности**. В нейронных сетях мы привыкли к **воспроизводимости** результата: в режиме валидации мы можем несколько раз подать на вход один и тот же объект и получить один и тот результат.  Возникает два вопроса:
- что подавать на **вход** сети для генерации?
- как реализовать **случайность**?

Ответ на оба вопроса: подавать в качестве **входа** вектор **случайного шума**.

<img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/generator_model_pipeline.png" width="700">

Почему именно **вектор**? Почему не одно **случайное число**? 

**Ответ**: входной вектор можно рассматривать, как **признаки** генерируемого объекта. Каждый такой признак - **независимая случайная величина**. Если мы будем передавать только одно случайное число, то генерация будет однообразной. Чем больше признаков (степеней свободы) у входного вектора, тем разнообразнее будет результат генерации.

То есть случайный шум **большей размерности** даёт нам **больше вариабельности**  для генерации. Это называется **input latent space** - входное латентное пространство.

**Note:** *из-за неустоявшейся терминологии случайное распределение на входе генератора называется латентным пространством, так же как и скрытое пространство в автоэнкодерах. Поэтому в этой лекции будем называть его **входным** латентным пространством. Также в статьях встречается вариант: predefined latent space.*

Каждую компоненту латентного пространства можно рассматривать как отдельную шкалу, вдоль которой изменяются определенные свойства генерируемых объектов. Например, можно выбрать четыре латентных вектора и посмотреть, как генерируемые объекты плавно изменяются при переходе от одного вектора к другому.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/input_latent_space_lin_interpol.png" width="600">

*Линейные интерполяции между четырьмя изображениями в латентном пространстве.*

[M. Pieters, M. Wiering](https://arxiv.org/abs/1803.09093)</center>

#### Размерность входного латентного пространства

В выборе размерности входного латентного пространства важно соблюсти  баланс.
- при **низкой размерности** возникнет проблема **низкой вариабельности**. 

Пример: генератор лиц с входным вектором длины 1. Результатом работы генератора будет всего одна шкала, вдоль которой будут расположены генерируемые изображения. Скорее всего, генератор выучит наиболее простую и "очевидную" шкалу - от молодой женщины блондинки к пожилому мужчине брюнету. У такой сети будет низкая вариабельность - она не сможет сгенерировать, например, рыжего ребенка в очках.

- при **большой размерности** латентного пространства, пространство может быть слишком **разреженным**. 

При обучении модели количество точек в этом латентном пространстве будет настолько мало, что в основном пространство будет состоять из пустот. Тогда модель будет крайне некачественно генерировать объекты в точках латентного пространства, далеких от точек обучающей выборки.

Лучший способ выбрать длину вектора - это найти публикацию с похожей задачей и взять значение из нее.

Если такой информации нет, то придется экспериментировать. Лучше начинать с низкой размерности латентного пространства, чтобы наладить работу всей сети, пусть и с низким разнообразием, а затем проводить эксперименты по поиску оптимальной размерности.

Можно использовать собственные знания в предметной области: спросите себя, сколькими вещественными числами можно описать важную информацию об объекте.

#### Распределение входных латентных векторов

Как мы знаем из лекции про обучение сети, инициализация весов и нормализация входных данных имеют существенный вклад в работу модели. Поэтому, принято использовать **многомерное нормальное распределение** для input latent space. Оно лучше взаимодействует с весами модели и улучшает сходимость.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/binomial_distribution.png" width="600"></center>

<center><em><a href="https://matplotlib.org/3.1.0/gallery/lines_bars_and_markers/scatter_hist.html">Двумерное нормальное распределение</a></p> </em></center>


## Наивный подход в решении задачи генерации
(как делать на практике НЕ нужно)

Давайте попробуем создать генератор точек на параболе. Самым тривиальным решением, кажется, подача случайного шума на вход сети и будем ожидать на выходе точки параболы. Проверим, как это будет работать.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def gen_pair(num=100):
    x = np.random.uniform(low=-1, high=1, size=(num,))
    y = x * x
    return np.hstack((x.reshape(-1, 1), y.reshape(-1, 1))) # Create num of correct dots(x,y) on parabola 

pairs = gen_pair(100)
plt.scatter(pairs[:, 0], pairs[:, 1])
plt.title("Random dots on parabola,\nwhich will use like a dataset.")
plt.show()

Возьмём размерность входного латентного пространства $ls = 1$ и объединим шум с точками в датасеты.

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Define input parameters 
n_batches = 10
batch_size = 128
ls = 1 # latent space

# Generate random noise
noise = torch.randn(size=(n_batches * batch_size, ls), dtype=torch.float)
print(f"NN Input: noise.shape: {noise.shape}")

# Generates dots on parabola
xy_pair = gen_pair(num=(n_batches * batch_size))
xy_pair = torch.tensor(xy_pair, dtype=torch.float)
print(f"NN Output: xy_pair.shape: {xy_pair.shape}")

dataset = TensorDataset(noise, xy_pair) # model inputs, model outputs
trainset, testset = train_test_split(dataset, train_size=0.8) # split dataset for train and test

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True)

Определим простую модель, которая будет ожидать шум на вход, и генерировать точки на выходе. (Обратите внимание, что функция активации на последнем слое отсутствует, поскольку мы не ограничиваем наш генератор в каком-то диапазоне.)

In [ ]:
import torch.nn as nn

class GenModel(nn.Module):
    def __init__(self, latent_space):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_space, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 2)) # x,y

    def forward(self, x):
        return self.model(x)

Напишем функцию для оценки лосса.

Так как мы не знаем, в каком месте параболы генератор создаст новую точку, то непонятно, с каким элементом из датасета ее сравнивать.

**Вариант №1**

Для сгенерированного $x$ аналитически вычислять $y_{target}=x*x$ и считать разницу между $y$ сгенерированным моделью и $y_{target}$ вычисленным аналитически:

In [ ]:
def custom_loss(pair, label):
  # All inputs are batches
  x_fake = pair[:, 0] 
  y_fake = pair[:, 1]
  return torch.abs(x_fake * x_fake - y_fake).mean() # average by batch

Это будет работать. 

Однако, если мы знаем способ точно предсказать выход по входу значит задача уже решена и нейронная сеть не нужна.

**Вариант №2**

Найти в датасете точку  $ target = (x_{target},y_{target})$ наиболее близкую к созданной генератором $ generated = (x,y)$ и использовать расстояние между этими точками качестве лосс. 

$$ Loss = min(dist(target_{i},generated))$$

В пространстве высокой размерности такой поиск будет весьма ресурсозатратным, но в нашем учебном примере работать будет.


In [ ]:
class Loss(nn.Module):
  def __init__(self, targets):
    super().__init__()
    self.targets = targets # Remember all real samples, impossible in real world

  def forward(self,input, dummy_target=None):
    dist = torch.cdist(input, self.targets) # claculate pairwise distances (euc.)
    min_dist, index = torch.min(dist, dim = 1) # take the best
    return min_dist.mean() 

Вспомогательный код для вывода loss 

In [ ]:
@torch.inference_mode()
def get_test_loss(model, loader):
    test_data = next(iter(loader))
    test_loss = Loss(test_data[1])
    outputs = model(samples.to(device))
    return test_loss(outputs.cpu())

Основной код обучения.

Целевые точки из датасета запоминаются в loss, затем идет обычный цикл обучения.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 600
model = GenModel(latent_space = ls)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

all_train_targets = next(iter(train_loader))[1]
criterion = Loss(all_train_targets.to(device))

for epoch in range(num_epochs):
    loss_epoch = 0
    for samples, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(samples.to(device))
        loss = criterion(outputs.to(device), labels.to(device))
        loss.backward()
        optimizer.step()
        loss_epoch += loss.item()
        
    loss_test = get_test_loss(model, test_loader)
    if epoch % 100 == 0:
        print(f"Epoch={epoch} train_loss={loss_epoch/len(train_loader):.4} test_loss={loss_test:.4}")

Посмотрим результаты генерации на шуме

In [ ]:
def test_image(model, pairs, ls=1):
    model.eval().to('cpu')
    noise = torch.tensor(np.random.normal(size=(1000, ls)), dtype=torch.float)
    with torch.no_grad():
        xy_pair_gen = model(noise)

    xy_pair_gen = xy_pair_gen.detach().numpy()
    plt.scatter(pairs[:, 0], pairs[:, 1], color='red', label='real')
    plt.scatter(xy_pair_gen[:, 0], xy_pair_gen[:, 1], color='blue', label='generated')
    plt.axis([-1, 1, 0, 1])
    plt.legend()
    plt.show()
    model.to(device)

test_image(model, pairs)

Видно, что модель генерирует точки, лежащие на параболе, при этом все они лежат в довольно узком интервале по оси х. 


Это неудивительно: в лосс - функции мы прописали, что сгенерированная точка должна лежать на параболе и модель обучилась. А информацию о том, в каких частях кривой должны оказаться точки, мы в loss никак не кодировали.

Более того модель может научиться хорошо генерировать одну единственную точку и при этом лосс может стать нулевым.


Итак, надо решить две проблемы:


1.   Закодировать в Loss условие о том, что точки должны быть различными
2.   Придумать способ проверки не требующий перебора всего датасета



## Дискриминатор

Будем наказывать нейронную сеть не напрямую стандартной лосс функцией, а второй **сетью**, которая определяет, лежит ли сгенерированная точка на параболе. 

Создадим сеть-классификатор точек (лежит/не лежит на параболе), которую назовём **дискриминатор** или критик.

In [ ]:
class DisModel(nn.Module):
    def __init__(self, n_points):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 25),
            nn.ReLU(),
            nn.Linear(25, 15),
            nn.ReLU(),
            nn.Linear(15, 1), # real/fake
            nn.Sigmoid())
        
    def forward(self, x):
        x = x.view(-1, 2 * n_points)
        return self.model(x)

В общем случае задача дискриминатора - определять, принадлежит ли объект к распределению обучающей выборки.

**Итого** мы имеем: 
- **генератор**, выдающий точки, которые могут принадлежать параболе, а могут не принадлежать ей
- **дискриминатор**, который будет их различать

Мы будем подавать в **дискриминатор** **правильные точки** (чтобы он знал, как это должно выглядеть), и **точки, которые выдаёт генератор**, считая их подделкой.

Таким образом, **генератор** будет учиться **подражать** реальным данным, а дискриминатор будет учиться **отличать** реальные точки, от подделок. 

Мы пришли к идее **генеративно-состязательных** нейронных сетей.

##  Generative adversarial network (GAN)

[2014 Generative Adversarial Networks (Goodfellow et al., 2014)](https://arxiv.org/abs/1406.2661) (**Cited by 33430!!!**)

[Видео разбор оригинальной статьи](https://youtu.be/eyxmSmjmNS0)

[Видео лекции Иана Гудфеллоу](https://www.youtube.com/watch?v=HGYYEUSm-0Q)

**Генеративно-состязательную** сеть описал Иан Гудфеллоу из компании Google (на тот момент) в 2014 году. Сейчас он возглавляет подразделение машинного обучения в Apple. Принцип состязательности в сети **GAN** нередко описывается через метафоры.

<p>
    <center><img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/generative_adversarial_network_scheme.png" width="700"></center>
    <center><em>Схематичное представление архитектуры GAN </em></center>
</p>




### Генератор - фальшивомонетчик!

Еще со времен **AlexNet** мы знаем, что если мы что-то и умеем делать с нейросетями - так это **классификаторы**. В классическом GAN **дискриминатор** выполняет простейшую из задач классификации - **бинарную классификацию** (либо *real*, либо *fake*). А вот задача **генерации** каким-то прямым образом на тот момент решена не была.

Как использовать всю мощь классификатора для создания генератора?

Представим, что есть фальшивомонетчик $G$ (generator) и банкир с прибором для проверки подлинности купюр $D$ (discriminator).

Фальшивомонетчик черпает вдохновение из генератора случайных чисел в виде случайного шума $z$ и создает подделки $G(z)$.

Банкир $D$ получает на вход пачку купюр $x$, проверяет их подлинность и сообщает вектор $D(x)$, состоящий из чисел от нуля до единицы - свою уверенность (вероятность) по каждой купюре в том, что она настоящая. Его цель - выдавать нули для подделок $D(G(z))$ и единицы для настоящих денег $D(x)$. Задачу можно записать как максимизацию произведения $D(x)(1-D(G(z)))$, а произведение, в свою очередь, можно представить как сумму через логарифм.

Таким образом, задача банкира - максимизировать $log(D(x))+log(1-D(G(z)))$.

Цель фальшивомонетчика прямо противоположна - максимизировать $D(G(z))$, то есть убедить банкира в том, что подделки настоящие.

Продолжая аналогию, обучение генератора можно представить так: фальшивомонетчик не просто генерирует подделки наудачу. Он добывает прибор для распознавания подделок, разбирает его, смотрит, как тот работает, и затем создает подделки, которые смогут обмануть этот прибор.

Математически - это **[игра](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B8%D1%8F_%D0%B8%D0%B3%D1%80) двух игроков**:

$$\min\limits_{\theta_g}  \max\limits_{\theta_d} [\mathbb{E}_{x _\sim p(data)} log(D_{\theta_d}(x)]+\mathbb{E}_{z _\sim p(z)} 
[log(1-D_{\theta_d}(G_{\theta_g}(z))]$$



**Дискриминатор** 
- обучается при **фиксированном генераторе** ${G}_{\theta_{g}}$,
- **максимизирует** функцию выше относительно $\theta_d$ (**градиентный подъем**),
- решает задачу **бинарной классификации**: старается присвоить $1$ точкам данных из обучающего набора $E_{x∼p_{data}}$ и 0 сгенерированным выборкам $E_{z∼p(z)}$.


**Генератор**
- обучается при **фиксированном дискриминаторе** $D_{θ_d}$,
- получает градиенты весов за счет backpropagation через дискриминатор,
- **минимизирует** функцию выше относительно $\theta_d$ (**градиентный спуск**).

Посредством **чередования** градиентного **подъема** и **спуска** сеть можно обучить.

Градиентный **подъем** на **дискриминаторе**:


$$\max\limits_{\theta_d} [\mathbb{E}_{x _\tilde{}p(data)} log(D_{\theta_d}(x)+\mathbb{E}_{z _\tilde{}p(z)} log(1-D_{\theta_d}(G_{\theta_g}(z)))]$$

Градиентный **спуск** на **генераторе**:


$$\min\limits_{\theta_g} \mathbb{E}_{z _\tilde{}p(z)} log(1-D_{\theta_d}(G_{\theta_d}(z)))$$

Градиентный **спуск** на **генераторе** эквивалентен градиентному **подъему**

$$\max\limits_{\theta_g} \mathbb{E}_{z _\tilde{}p(z)} log(D_{\theta_d}(G_{\theta_d}(z)))$$

В процессе совместного конкурентного обучения, если система достаточно сбалансирована, достигается **минимаксное состояние равновесия**, в котором обе сети эффективно учатся.

Сгенерированные удачно обученной нейросетью изображения практически неотличимы от настоящих.

Если хорошенько подумать - то можно прийти к выводу, что **loss-функция** в **GAN** это не какая-то функция заданная людьми, а еще одна **нейросеть**.

**Преимущества GAN**
* Теоретические **гарантии сходимости**
* Можно обучать обычным **SGD/Adam**
* Решает в явном виде задачу **generative modeling**, но неявным образом (**нейросети**)

**Недостатки GAN**
* **Нестабильное обучение**
* Очень **долгая сходимость**
* **Mode-collapsing** (модель выдает одно и то же изображение или один и тот же класс и т.д., независимо от того, какие входные данные ей подаются)
* **Исчезновение градиента**: дискриминатор настолько хорошо научился отличать сгенерированные образцы от реальных, что градиент весов генератора становится равным 0: в какую сторону бы генератор не изменил свои веса, дискриминатор все равно идеально распознает фальшивки
* Поиск оптимальных параметров - **pure luck**

### GAN Практический пример

Определим наш **генератор** и **дискриминатор**

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_space, hidden_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_space, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)) # x,y

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1), # real/fake
            nn.Sigmoid())
        
    def forward(self, x):
        return self.model(x)

Определим **входные параметры**

In [ ]:
latent_dim = 10 # latent space 
num_epochs = 10000
batch_size = 32

device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

Обратите внимание, что у нас, так же как и в первом примере, есть переменная **latent space**. Это тот шум, из которого мы будем генерировать наши точки. Закон сохранения масс в действии - нельзя создать что-то из ничего!

Определим все необходимое для обучения

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

criterion = nn.BCELoss() # Loss
gen = Generator(latent_space=latent_dim, hidden_dim=50).to(device)
disc = Discriminator(hidden_dim=50).to(device)

# 2 optimizers for Discriminator and Generator 
optimizerD = torch.optim.Adam(disc.parameters(), lr=3e-4)
optimizerG = torch.optim.Adam(gen.parameters(), lr=3e-4)

# Fix noise to compare
fixed_noise = torch.randn(128, latent_dim, device=device) 

Обратите внимание, что мы используем `BCELoss` (**Binary Cross Entropy**). Давайте разберемся почему:

- **Дискриминатор** решает задачу **бинарной классификации**. Для этой задачи хорошо подходит **BCE**.
- Требование к генератору может быть сформулировано как "объектам сгенерированным **генератором**, **дискриминатором** должна быть присвоена **высокая вероятность**". Для **"идеального" генератора**, который всегда генерирует фотореалистичные результаты, значения **$D(G(z))$** всегда должны быть **близки к 1**. Для этой задачи хорошо подходит **BCE**.

Определим функцию, которая создает точки на параболе

In [ ]:
def gen_pair(num=100):
    x = np.random.uniform(low=-1, high=1, size=(num,))
    y = x * x
    return torch.tensor(np.hstack((x.reshape(-1, 1), y.reshape(-1, 1))), dtype=torch.float) # Create num of correct dots(x,y) on parabola 

Что сейчас будет происходить? $$$$

* Обучение дискриминатора
    * обнулим градиенты **дискриминатора**
    * real точки
        * создадим набор **real точек**, которые лежат на параболе
        * посчитаем лосс дискриминатора на **real точках** и **real метках** $\text{loss D}_\text{real}$
        * посчитаем градиенты для **дискриминатора**

    <img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/gan_training_algorithm_1.png" width="1000">

    * fake точки
        * сгенерируем случайный шум $z$
        * возьмем наш не обученный **генератор** и создадим с его помощью **fake точки** из $z$
        * посчитаем лосс дискриминатора на **fake точках** и **fake метках** $\text{loss D}_\text{fake}$
        * посчитаем градиенты для **дискриминатора** (они сложатся с уже посчитанными ранее)
    * обновление весов 
        * сделаем шаг обучения **дискриминатора** (обвновим его веса)
        * **генератор** не обучается

    <img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/gan_training_algorithm_2.png" width="1000">

* Обучение генератора
    * обнулим градиенты **генератора**
    * сгенерируем случайный шум $z$
    * создадим с помощью **генератора** набор **fake точек** из $z$
    * посчитаем значение функции потерь дискриминатора на **fake точках** и **real метках** $\text{loss G}$ (подмена меток)
    * посчитаем градиенты для **генератора**
    * сделаем шаг обучения **генератора** (обвновим его веса)
    * **дискриминатор** не обучается

    <img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/gan_training_algorithm_3.png" width="1000">

Смотрите код внимательно, чтобы понять, о чем речь.

In [ ]:
from IPython import display

real_label = 1
fake_label = 0

#Main Training Loop
print("Training...")
print(device)

x = []
y_D = []
y_G = []
for epoch in range(num_epochs):
      #max log(D(x)) + log(1 - D(G(z)))
      #train on real points
      disc.zero_grad()
      
      # Define real points
      real_points = gen_pair(num=batch_size).to(device)
      label = torch.full((batch_size,), real_label, dtype=torch.float, device=device).view(-1)
      
      # Train disc on real_points
      output = disc(real_points).view(-1)
      errD_real = criterion(output, label)
      errD_real.backward()

      # Define fake points
      # This dots generated by generator transform from latent space 
      noise = torch.randn(batch_size, latent_dim, device=device)
      fake_points = gen(noise)
      label.fill_(fake_label)

      # Train disc on fake_points 
      output = disc(fake_points.detach()).view(-1)
      errD_fake = criterion(output, label)
      errD_fake.backward()

      # Discriminator loss(real+fake)
      errD = errD_real + errD_fake

      optimizerD.step()

      #max log(D(G(z)))
      # Now, train generator
      gen.zero_grad()

      # Let's tell the discriminator that our generator creates real points
      label.fill_(real_label)

      output = disc(fake_points).view(-1)

      errG = criterion(output, label)

      errG.backward()

      optimizerG.step()

      # Plotting every N epoch 
      x.append(epoch)
      y_D.append(errD.item() / 2)
      y_G.append(errG.item())
      
      if epoch % 250 == 0:
        fig, ax = plt.subplots(nrows=2)
        ax[0].plot(x, y_D, color='red', lw=1, label='D')
        ax[0].plot(x, y_G, color='green', lw=1, label='G')

        # Generates dots from fixed_noise 
        fake_points = gen(fixed_noise)
        ax[1].scatter(fake_points.detach().to('cpu')[:, 0], fake_points.detach().to('cpu')[:, 1], color='green', alpha=0.5)
        ax[1].scatter(real_points.detach().to('cpu')[:, 0], real_points.detach().to('cpu')[:, 1], color='red', alpha=0.5)
        ax[1].set_xlim(-1, 1)
        ax[1].set_ylim(0, 1)
        display.clear_output(wait=True)
        display.display(plt.gcf())
        plt.close()

**Класс!** У нас получилось (если вдруг не сошлось за 10000 эпох, перезапустите заново, к сожалению, **фиксация seed еще не гарантирует стабильность GAN**). Особенно круто смотреть как красиво лосс **дискриминатора** и **генератора** сходятся.

## DCGAN — Генерация изображений

С помощью **GAN** можно, разумеется, генерировать не только точки на параболе. Можно генерировать, например, изображения. Но поднимаются закономерные вопросы.

### Как из шума на входе сети получить изображение?

Самым простым ответом будет: взять шум, пропустить его через **полносвязные слои** и сделать **reshape** до нужного разрешения. В целом, это будет работать.


Однако **DCGAN - Deep Convolutional GAN** использует **сверточные** и **сверточно-транспонированные** (*convolutional* и *convolutional-transpose*) слои в дискриминаторе и генераторе соответственно. Впервые метод **DCGAN** был описан в статье [Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks (Radford et al., 2015)](https://arxiv.org/abs/1511.06434).

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/deep_convolutional_gan_scheme.png" width="700"></center>
<center><em>Схема работы DCGAN (Radford et al., 2015).</em></center>

Ниже видна разница в генерации при помощи исключительно **полносвязных слоёв** и при помощи **обратных свёрток**. Очевидно, результат **DCGAN** лучше, чем **GAN**.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/gan_dcgan_mnist_examples.png" width="600"></center>
<center><em>Сравнение результатов на MNIST (Radford et al., 2015)</em></center>

### Архитектура DCGAN

На вход генератора подают шум для создания разнообразных объектов. Этот шум представляет собой вектор в многомерном пространстве. Один вектор - один сгенерированный объект. Задача дискриминатора - преобразовать вектор в изображение.

Такое преобразование возможно при помощи транспонированных сверточных (convolution-transpose, иногда называют fractionally strided convolution) слоев. Как и обычные сверточные слои, эти слои используют сверточные ядра, но перед вычислением сверток они увеличивают размер исходного изображения, "раздвигая" пиксели и заполняя образующиеся промежутки между пикселями нулями.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/dcgan_architecture.png" width="700"></center>
<center><em>Зеркальная архитектура DCGAN </em></center>

### Convolution-Transpose Layer

Давайте кратко вспомним, что делают CT слои?

Как работают **транспонированные сверточные** слои хорошо демонстрирует рисунок ниже. Между точками исходной карты признаков (темно синие квадраты) вставляется сетка из нулей (голубые квадраты) после чего над полученной разреженной матрицей признаков совершается обычная **свертка** с шагом 1. 

Из изображения 3х3 в изображение 5х5 с использованием паддинга (Padding, strides, transposed):

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/convolution_transpose_layer.gif" </center>
<center><em>Из изображения 3х3 в изображение 5х5 с использованием паддинга (Padding, strides, transposed).</em></center>

[Подробнее можно почитать тут](https://github.com/vdumoulin/conv_arithmetic)

In [ ]:
x = torch.rand((1, 3, 10, 10)) * 255 # one 3-channel image with 10x10 size
print(x.shape)

In [ ]:
convT = nn.ConvTranspose2d(in_channels=3, out_channels=3, kernel_size=3)
y = convT(x)
print(y.shape)  # One 3-chanells image with 12x12 size

Полученное изображение не похоже на входное - потому что были применены свёрточные ядра со случайными коэффициентами.

In [ ]:
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True)
ax[0].imshow(x[0].permute(1, 2, 0).detach().numpy().astype(np.uint8))
ax[1].imshow(y[0].permute(1, 2, 0).detach().numpy().astype(np.uint8))
ax[0].set_title('Input')
ax[1].set_title('After ConvTranspose')
plt.show()

### Другие способы повышения разрешения  - Upsampling

Помимо **обратных свёрток**, существует другие методы повышения разрешения из низкой размерности.

Самый простой способ - выполнить повышение разрешения с помощью **интерполяции**. Давайте вспомним, что в PyTorch это осуществляется слоем [Upsample](https://pytorch.org/docs/stable/generated/torch.nn.Upsample.html)



In [ ]:
x = torch.rand((1, 3, 10, 10)) # one 3-channal image with 10x10 size
print("Input shape:", x.shape)
    
upsample = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)
y = upsample(x)

print("Output shape", y.shape)

In [ ]:
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True)
ax[0].imshow((x[0].permute(1, 2, 0) * 256).detach().numpy().astype(np.uint8))
ax[1].imshow((y[0].permute(1, 2, 0).detach().numpy() * 256).astype(np.uint8))
ax[0].set_title('Input')
ax[1].set_title('After Upsample')
plt.show()

### Пример обученного DCGAN

Давайте посмотрим на пример обученного **DCGAN**

In [ ]:
from IPython.display import clear_output
use_gpu = True if torch.cuda.is_available() else False
model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub', 'DCGAN', pretrained=True, useGPU=use_gpu)
clear_output()

In [ ]:
import torchvision
from warnings import simplefilter
simplefilter("ignore", category=UserWarning)

num_images = 16
noise, _ = model.buildNoiseData(num_images)
with torch.no_grad():
    generated_images = model.test(noise)

fig, ax = plt.subplots(figsize=(16 * 3, 2 * 3))
ax.imshow(torchvision.utils.make_grid(generated_images).permute(1, 2, 0).cpu().numpy(), 
          interpolation='nearest', aspect='equal')
ax.axis('off')
plt.show()

### Практический пример DCGAN

Теперь давайте попробуем сами написать свой **DCGAN** и обучить его на датасете **FashionMNIST**

In [ ]:
num_epochs = 2        # Num of epochs
batch_size = 64       # batch size
lr = 2e-4             # Learning rate
b1 = 0.5              # Adam: decay of first order momentum of gradient
b2 = 0.999            # Adam: decay of first order momentum of gradient
num_cpu = 8           # Num of cpu threads to generate batch 
latent_dim = 100      # latent space
img_size = 32         # images size
channels = 1          # Num of channels 
sample_interval = 450 # interval between image sampling

Обычно мы **инициализируем веса** случайным образом, но ничто не мешает нам инициализировать их так, как мы хотим. В [оригинальной статье](https://arxiv.org/pdf/1511.06434.pdf) про **DCGAN** предложено инициализировать веса нормальным распределением с центром в нуле и стандартным отклонением 0,02:

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

Обратите внимание, как преобразуется **шум** в **генераторе**:
* Сначала с помощью **полносвязного слоя** он преобразуется в **первичные фичи**
* Потом с помощью функции **view**, **ресэмплится** в картинку низкого разрешения
* Потом, проходя через **conv_blocks** поочерёдно применяются **Upsample** и **ОБЫЧНЫЕ** свёртки

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = img_size // 4
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity

In [ ]:
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

# Initialize Generator and Discriminator 
generator = Generator()
discriminator = Discriminator()

# Initialize weight
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Define loss
criterion = nn.BCELoss()

generator.to(device)
discriminator.to(device)
criterion.to(device)

Напишем функцию для отображения изображений

In [ ]:
from torchvision.utils import make_grid

def show_gen_img(model, latent_dim=100):
    z = Tensor(np.random.normal(0, 1, (9, latent_dim))) # define latent dim
    
    # Generate noise from latent dim 
    sample_images = generator(z) 
    sample_images = sample_images.cpu().detach()

    # Plotting images
    grid = make_grid(sample_images, nrow=3, ncols=3, normalize=True).permute(1, 2, 0).numpy()
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(grid)
    plt.axis('off')
    return fig

Подгрузим данные и загрузим и их в Data Loader

In [ ]:
import os
from torchvision import datasets, transforms

os.makedirs("../../data/mnist", exist_ok=True)
data_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)
clear_output()

In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor

for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(data_loader):

        # Adversarial ground truths
        valid = Tensor(imgs.shape[0], 1).fill_(1.0)
        fake = Tensor(imgs.shape[0], 1).fill_(0.0)

        # Configure input
        real_imgs = imgs.type(Tensor)

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim)))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = criterion(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = criterion(discriminator(real_imgs), valid)
        fake_loss = criterion(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        batches_done = epoch * len(data_loader) + i
        if batches_done % sample_interval == 0:
            print("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, num_epochs, i, len(data_loader), d_loss.item(), g_loss.item()))
            fig = show_gen_img(generator)
            plt.show()       

Чтобы картинки обрели приличный вид, хватает 2 эпох. Чтобы стали выглядеть хорошо - 5 эпох.

## cGAN — GAN с условием

**cGAN** расшифровывается как **Conditional Generative Adversarial Net** - это **GAN** с условием. Условие может быть любым, например, генерация конкретной цифры. В этом случае нам нужен уже размеченный датасет, для того чтобы обучить дискриминатор.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/conditional_gan_scheme.png" width="800"></center>
<center><em>Схема работы cGAN. Label Y добавляется к случайному шуму, тем самым мы говорим генератору генерировать случайное изображение нужного класса. Так же он подаётся в дискриминатор в качестве входа, чтобы дискриминатор знал какое изображение классифицировать как реальное, а какое как вымышленное.</em></center>

Обучение в данном случае будет аналогичным обучению **GAN**, мы будем обучать сети, чередуя реальные данные и сгенерированные, добавив `label`.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/cGANS_results_20_and_50_epochs_mnist.png" width="600"></center>
<center><em>Сравнение результатов cGAN и cDCGAN.</em></center>

### Как закодировать метки?

Поскольку подавать в сеть числа от 0 до 9 (в случае **MNSIT**) нет смысла, то нужно придумать, как подавать их в нейронную сеть. На помощь приходят **Embeddings**. Мы можем представить каждую метку в виде вектора с десятью элементами.

[Документация nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [ ]:
samples, labels = next(iter(data_loader))

label_emb = nn.Embedding(10, 10)

e = label_emb(labels)

print(f"Label: {labels[0]}")
print(f"Embedding for this label: {e[0]}")

После чего, **эмбединги** меток обычно склеиваются с входами сетей.

#### Почему нельзя подать просто число?

Если мы будем подавать просто число, например 0.1 для единицы, и 0.5 для пяти, то вход у нас будет непрерывным, что довольно нелогично, тогда при небольшом изменении входа, мы будем генерировать другую цифру. А так же потому что сети будет сложнее выучить небольшие расхождения в этом небольшом интервале. В случае с векторным представлением мы избегаем этих проблем.

#### Модификации cGAN

Метки классов можно подавать не только способом, описанным выше. Можно вместо подачи их в дискриминатор сделать так, чтобы он их предсказывал - **Semi-Supervized GAN**.

Или же не подавать label в дискриминатор, но ждать от него классификации в соответствии с классом, который мы хотим получить от генератора - это **InfoGAN**

Ещё одна модификация cGAN - это **AC-GAN** (auxiliary classifier) в которой единственное различие заключается в том, что дискриминатор должен помимо распознавания реальных и фейковых изображений ещё и классифицировать их. Он имеет эффект стабилизации процесса обучения и позволяет генерировать большие высококачественные изображения, изучая представление в скрытом пространстве, которое не зависит от метки класса.

<img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/gans_zoo_schemes.png" width="900">

## ProGAN, StyleGAN, StyleGAN2, Alias-Free GAN

[2017 Progressive Growing of GANs for Improved Quality, Stability, and Variation (ProGAN) [Karras et al., 2017]](https://arxiv.org/abs/1710.10196)

[2018 A Style-Based Generator Architecture for Generative Adversarial Networks (StyleGAN) [Karras et al., 2018]](https://arxiv.org/abs/1812.04948)

[2019 Analyzing and Improving the Image Quality of StyleGAN (StyleGAN2) [Karras et al., 2019]](https://arxiv.org/abs/1912.04958)

[2021 Alias-Free Generative Adversarial Networks (Alias-Free GAN) [Karras et al., 2021]](https://arxiv.org/abs/2106.12423)

На момент обновления блокнота (лето 2022 года), топовой архитектурой **GAN** для генерации изображений считается **Alias Free GAN**. Это можно посмотреть, например, [тут](https://paperswithcode.com/task/image-generation). В большинстве соревнований выигрывает **StyleGAN-XL** - обученный на картинки большого размера **Alias Free GAN**.

Для того что бы понять как он работает, давайте проследим за эволюцией работ от коллектива авторов из NVIDIA.

### ProGAN

**ProGAN**, или **Progressively Growing GAN** - это генеративная состязательная сеть, призванная улучшить качество генерации изображений с помощью добавления слоёв во время генерации. Сначала сеть учится генерировать **изображения низкого разрешения**, потом добавляется слой, который создаёт **более высокое разрешение** из фич предыдущего слоя и так далее. 

Другими словами, вместо того, чтобы пытаться обучить сразу все слои генератора и дискриминатора, как это обычно делается, авторы статьи постепенно **наращивали** свой **GAN**, по одному слою за раз, для обработки все более высоких разрешений изображений.


Эта идея позволяет **плавно увеличивать размер** изображения и добиваться **лучшей сходимости**.


<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/pro_gan_how_it_works.gif" width="600"></center>
<center><em>Принцип работы ProGAN (Karras et al., 2017)</em></center>

Для того чтобы все получилось авторы сначала искусственно уменьшили свои обучающие изображения до очень **маленького разрешения** (всего **4x4** пикселя). Они создали **генератор** с несколькими слоями для синтеза изображений с таким низким разрешением и соответствующий **дискриминатор** с зеркальной архитектурой. Поскольку эти сети были такими маленькими, они обучались относительно быстро, но усваивали только **крупномасштабные структуры**, видимые на сильно размытых изображениях.

Когда первые слои завершили обучение, они добавили еще один слой к **G** и **D**, удвоив выходное разрешение до **8x8**. **Обученные веса** в предыдущих слоях сохранялись, но **не фиксировались**. 

Обучение продолжалось до тех пор, пока **GAN** снова не стал синтезировать убедительные изображения, на этот раз в новом разрешении **8x8**.

Таким образом, они продолжали добавлять слои, удваивать разрешение и тренироваться до тех пор, пока не был достигнут **желаемый размер** выходного сигнала.

Обучать мы его, конечно же, не будем, но поиграться с обученной моделью можем

In [ ]:
use_gpu = True if torch.cuda.is_available() else False

# load a model trained on a celebrity dataset "celebA"
# resolution of generated images 512 x 512 pixel
model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub',
                       'PGAN', model_name='celebAHQ-512',
                       pretrained=True, useGPU=use_gpu)

Попробуем что-нибудь сгенерировать

In [ ]:
num_images = 4
noise, _ = model.buildNoiseData(num_images)
with torch.no_grad():
    generated_images = model.test(noise)

plt.figure(figsize=(20, 10))
grid = torchvision.utils.make_grid(generated_images.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.permute(1, 2, 0).to('cpu').numpy())
plt.axis('off')
plt.show()

### StyleGAN

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/style_gan_scheme.png" width="400"></center>
<center><em>Сравнение архитектуры ProGAN и StyleGAN (<a href="https://arxiv.org/pdf/1812.04948.pdf">Karras et al., 2018</a>)</p> </em></center>

Идея с **ProGAN** оказалась хорошей. В 2018 году, команде исследователей из **NVIDIA** (все той же что и в **ProGAN**) пришла в голову мысль как процесс обучения **GAN** можно было бы сделать еще лучше. Они представили **StyleGAN** - расширение **ProGAN**. В дополнение к инкрементальному **росту моделей** во время обучения, **Style GAN** значительно изменяет **архитектуру** самого **генератора**.

Обычный **генератор** в **классическом GAN** работает так - берем **шум**, пропускаем его через некую обученную сеть-генератор и получаем **изображение** на выходе. Генератор же **StyleGAN** больше **не берет** на вход **вектор из латентного пространства**; вместо этого для создания синтетического изображения используются **два** новых **источника случайности**: отдельная картирующая сеть (**Mapping Network**) и добавление **гауссовского шума** к выходам слоев.

На выходе из **Mapping Network** получают вектор, определяющий стили, который интегрируется в каждую точку модели генератора с помощью нового слоя, называемого **слоем адаптивной нормализации** (Adaptive Instance Normalization или **AdaIN**). Использование этого вектора стилей дает контроль над стилем генерируемого изображения.

Разберемся подробнее с тем, как работают стили. Вектор стилей состоит из пар чисел $(y_{s,i}, y_{b,i})$, которые в блоке **AdaIN** модифицируют слои карты признаков по следующей формуле (нормировка и линейное преобразование): 
$$AdaIN(x_i, y_i) = \frac{x_i \mu(x_i)}{σ(x_i)}+y_{b,i} $$

Для каждого блока **AdaIN** для каждого слоя карты признаков предназначена своя пара чисел. Вектор стиля генерируется для всех слоев одновременно, это позволяет обеспечить лучшую согласованность между признаками разной сложности. 


<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/stylegan_style.png" width="750"></center>

<center><em>Пример смешения стилей двух изображений. Изображения поделены на две группы A и B. Группа A - левый столбик, группа B - верхняя строка. При генерации остальных изображений, часть вектора стиля изображения из A заменяли на часть вектора стиля изображения из B. В верхних изображениях, были заменены векторы стиля, отвечающие за преобразование самых ранних слоев, отвечающих за общий силуэт изображения (форма и положение головы, пол и возраст). Ниже даны изображения, где заменили средние слои (на них пол и возраст сохраняется, при этом меняются черты лица). В последней строке заменили финальную часть вектора стиля (это повлияло только на цветовую гамму). (<a href="https://arxiv.org/pdf/1812.04948.pdf">Karras et al., 2018</a>)</p> </em></center>

In [ ]:
#@title run StyleGAN

import torch.nn.functional as F
from collections import OrderedDict
import pickle

class MyLinear(nn.Module):
    """Linear layer with equalized learning rate and custom learning rate multiplier."""
    def __init__(self, input_size, output_size, gain=2**(0.5), use_wscale=False, lrmul=1, bias=True):
        super().__init__()
        he_std = gain * input_size**(-0.5) # He init
        # Equalized learning rate and custom learning rate multiplier.
        if use_wscale:
            init_std = 1.0 / lrmul
            self.w_mul = he_std * lrmul
        else:
            init_std = he_std / lrmul
            self.w_mul = lrmul
        self.weight = torch.nn.Parameter(torch.randn(output_size, input_size) * init_std)
        if bias:
            self.bias = torch.nn.Parameter(torch.zeros(output_size))
            self.b_mul = lrmul
        else:
            self.bias = None

    def forward(self, x):
        bias = self.bias
        if bias is not None:
            bias = bias * self.b_mul
        return F.linear(x, self.weight * self.w_mul, bias)

class MyConv2d(nn.Module):
    """Conv layer with equalized learning rate and custom learning rate multiplier."""
    def __init__(self, input_channels, output_channels, kernel_size, gain=2**(0.5), use_wscale=False, lrmul=1, bias=True,
                intermediate=None, upscale=False):
        super().__init__()
        if upscale:
            self.upscale = Upscale2d()
        else:
            self.upscale = None
        he_std = gain * (input_channels * kernel_size ** 2) ** (-0.5) # He init
        self.kernel_size = kernel_size
        if use_wscale:
            init_std = 1.0 / lrmul
            self.w_mul = he_std * lrmul
        else:
            init_std = he_std / lrmul
            self.w_mul = lrmul
        self.weight = torch.nn.Parameter(torch.randn(output_channels, input_channels, kernel_size, kernel_size) * init_std)
        if bias:
            self.bias = torch.nn.Parameter(torch.zeros(output_channels))
            self.b_mul = lrmul
        else:
            self.bias = None
        self.intermediate = intermediate

    def forward(self, x):
        bias = self.bias
        if bias is not None:
            bias = bias * self.b_mul
        
        have_convolution = False
        if self.upscale is not None and min(x.shape[2:]) * 2 >= 128:
            # this is the fused upscale + conv from StyleGAN, sadly this seems incompatible with the non-fused way
            # this really needs to be cleaned up and go into the conv...
            w = self.weight * self.w_mul
            w = w.permute(1, 0, 2, 3)
            # probably applying a conv on w would be more efficient. also this quadruples the weight (average)?!
            w = F.pad(w, (1,1,1,1))
            w = w[:, :, 1:, 1:]+ w[:, :, :-1, 1:] + w[:, :, 1:, :-1] + w[:, :, :-1, :-1]
            x = F.conv_transpose2d(x, w, stride=2, padding=(w.size(-1)-1)//2)
            have_convolution = True
        elif self.upscale is not None:
            x = self.upscale(x)
    
        if not have_convolution and self.intermediate is None:
            return F.conv2d(x, self.weight * self.w_mul, bias, padding=self.kernel_size//2)
        elif not have_convolution:
            x = F.conv2d(x, self.weight * self.w_mul, None, padding=self.kernel_size//2)
        
        if self.intermediate is not None:
            x = self.intermediate(x)
        if bias is not None:
            x = x + bias.view(1, -1, 1, 1)
        return x

class NoiseLayer(nn.Module):
    """adds noise. noise is per pixel (constant over channels) with per-channel weight"""
    def __init__(self, channels):
        super().__init__()
        self.weight = nn.Parameter(torch.zeros(channels))
        self.noise = None
    
    def forward(self, x, noise=None):
        if noise is None and self.noise is None:
            noise = torch.randn(x.size(0), 1, x.size(2), x.size(3), device=x.device, dtype=x.dtype)
        elif noise is None:
            # here is a little trick: if you get all the noiselayers and set each
            # modules .noise attribute, you can have pre-defined noise.
            # Very useful for analysis
            noise = self.noise
        x = x + self.weight.view(1, -1, 1, 1) * noise
        return x

class StyleMod(nn.Module):
    def __init__(self, latent_size, channels, use_wscale):
        super(StyleMod, self).__init__()
        self.lin = MyLinear(latent_size,
                            channels * 2,
                            gain=1.0, use_wscale=use_wscale)
        
    def forward(self, x, latent):
        style = self.lin(latent) # style => [batch_size, n_channels*2]
        shape = [-1, 2, x.size(1)] + (x.dim() - 2) * [1]
        style = style.view(shape)  # [batch_size, 2, n_channels, ...]
        x = x * (style[:, 0] + 1.) + style[:, 1]
        return x

class PixelNormLayer(nn.Module):
    def __init__(self, epsilon=1e-8):
        super().__init__()
        self.epsilon = epsilon
    def forward(self, x):
        return x * torch.rsqrt(torch.mean(x**2, dim=1, keepdim=True) + self.epsilon)

class BlurLayer(nn.Module):
    def __init__(self, kernel=[1, 2, 1], normalize=True, flip=False, stride=1):
        super(BlurLayer, self).__init__()
        kernel=[1, 2, 1]
        kernel = torch.tensor(kernel, dtype=torch.float32)
        kernel = kernel[:, None] * kernel[None, :]
        kernel = kernel[None, None]
        if normalize:
            kernel = kernel / kernel.sum()
        if flip:
            kernel = kernel[:, :, ::-1, ::-1]
        self.register_buffer('kernel', kernel)
        self.stride = stride
    
    def forward(self, x):
        # expand kernel channels
        kernel = self.kernel.expand(x.size(1), -1, -1, -1)
        x = F.conv2d(
            x,
            kernel,
            stride=self.stride,
            padding=int((self.kernel.size(2)-1)/2),
            groups=x.size(1)
        )
        return x

def upscale2d(x, factor=2, gain=1):
    assert x.dim() == 4
    if gain != 1:
        x = x * gain
    if factor != 1:
        shape = x.shape
        x = x.view(shape[0], shape[1], shape[2], 1, shape[3], 1).expand(-1, -1, -1, factor, -1, factor)
        x = x.contiguous().view(shape[0], shape[1], factor * shape[2], factor * shape[3])
    return x

class Upscale2d(nn.Module):
    def __init__(self, factor=2, gain=1):
        super().__init__()
        assert isinstance(factor, int) and factor >= 1
        self.gain = gain
        self.factor = factor
    def forward(self, x):
        return upscale2d(x, factor=self.factor, gain=self.gain)

class G_mapping(nn.Sequential):
    def __init__(self, nonlinearity='lrelu', use_wscale=True):
        act, gain = {'relu': (torch.relu, np.sqrt(2)),
                     'lrelu': (nn.LeakyReLU(negative_slope=0.2), np.sqrt(2))}[nonlinearity]
        layers = [
            ('pixel_norm', PixelNormLayer()),
            ('dense0', MyLinear(512, 512, gain=gain, lrmul=0.01, use_wscale=use_wscale)),
            ('dense0_act', act),
            ('dense1', MyLinear(512, 512, gain=gain, lrmul=0.01, use_wscale=use_wscale)),
            ('dense1_act', act),
            ('dense2', MyLinear(512, 512, gain=gain, lrmul=0.01, use_wscale=use_wscale)),
            ('dense2_act', act),
            ('dense3', MyLinear(512, 512, gain=gain, lrmul=0.01, use_wscale=use_wscale)),
            ('dense3_act', act),
            ('dense4', MyLinear(512, 512, gain=gain, lrmul=0.01, use_wscale=use_wscale)),
            ('dense4_act', act),
            ('dense5', MyLinear(512, 512, gain=gain, lrmul=0.01, use_wscale=use_wscale)),
            ('dense5_act', act),
            ('dense6', MyLinear(512, 512, gain=gain, lrmul=0.01, use_wscale=use_wscale)),
            ('dense6_act', act),
            ('dense7', MyLinear(512, 512, gain=gain, lrmul=0.01, use_wscale=use_wscale)),
            ('dense7_act', act)
        ]
        super().__init__(OrderedDict(layers))
        
    def forward(self, x):
        x = super().forward(x)
        # Broadcast
        x = x.unsqueeze(1).expand(-1, 18, -1)
        return x

class Truncation(nn.Module):
    def __init__(self, avg_latent, max_layer=8, threshold=0.7):
        super().__init__()
        self.max_layer = max_layer
        self.threshold = threshold
        self.register_buffer('avg_latent', avg_latent)
    def forward(self, x):
        assert x.dim() == 3
        interp = torch.lerp(self.avg_latent, x, self.threshold)
        do_trunc = (torch.arange(x.size(1)) < self.max_layer).view(1, -1, 1)
        return torch.where(do_trunc, interp, x)

class LayerEpilogue(nn.Module):
    """Things to do at the end of each layer."""
    def __init__(self, channels, dlatent_size, use_wscale, use_noise, use_pixel_norm, use_instance_norm, use_styles, activation_layer):
        super().__init__()
        layers = []
        if use_noise:
            layers.append(('noise', NoiseLayer(channels)))
        layers.append(('activation', activation_layer))
        if use_pixel_norm:
            layers.append(('pixel_norm', PixelNorm()))
        if use_instance_norm:
            layers.append(('instance_norm', nn.InstanceNorm2d(channels)))
        self.top_epi = nn.Sequential(OrderedDict(layers))
        if use_styles:
            self.style_mod = StyleMod(dlatent_size, channels, use_wscale=use_wscale)
        else:
            self.style_mod = None
    def forward(self, x, dlatents_in_slice=None):
        x = self.top_epi(x)
        if self.style_mod is not None:
            x = self.style_mod(x, dlatents_in_slice)
        else:
            assert dlatents_in_slice is None
        return x


class InputBlock(nn.Module):
    def __init__(self, nf, dlatent_size, const_input_layer, gain, use_wscale, use_noise, use_pixel_norm, use_instance_norm, use_styles, activation_layer):
        super().__init__()
        self.const_input_layer = const_input_layer
        self.nf = nf
        if self.const_input_layer:
            # called 'const' in tf
            self.const = nn.Parameter(torch.ones(1, nf, 4, 4))
            self.bias = nn.Parameter(torch.ones(nf))
        else:
            self.dense = MyLinear(dlatent_size, nf*16, gain=gain/4, use_wscale=use_wscale) # tweak gain to match the official implementation of Progressing GAN
        self.epi1 = LayerEpilogue(nf, dlatent_size, use_wscale, use_noise, use_pixel_norm, use_instance_norm, use_styles, activation_layer)
        self.conv = MyConv2d(nf, nf, 3, gain=gain, use_wscale=use_wscale)
        self.epi2 = LayerEpilogue(nf, dlatent_size, use_wscale, use_noise, use_pixel_norm, use_instance_norm, use_styles, activation_layer)
        
    def forward(self, dlatents_in_range):
        batch_size = dlatents_in_range.size(0)
        if self.const_input_layer:
            x = self.const.expand(batch_size, -1, -1, -1)
            x = x + self.bias.view(1, -1, 1, 1)
        else:
            x = self.dense(dlatents_in_range[:, 0]).view(batch_size, self.nf, 4, 4)
        x = self.epi1(x, dlatents_in_range[:, 0])
        x = self.conv(x)
        x = self.epi2(x, dlatents_in_range[:, 1])
        return x


class GSynthesisBlock(nn.Module):
    def __init__(self, in_channels, out_channels, blur_filter, dlatent_size, gain, use_wscale, use_noise, use_pixel_norm, use_instance_norm, use_styles, activation_layer):
        # 2**res x 2**res # res = 3..resolution_log2
        super().__init__()
        if blur_filter:
            blur = BlurLayer(blur_filter)
        else:
            blur = None
        self.conv0_up = MyConv2d(in_channels, out_channels, kernel_size=3, gain=gain, use_wscale=use_wscale,
                                 intermediate=blur, upscale=True)
        self.epi1 = LayerEpilogue(out_channels, dlatent_size, use_wscale, use_noise, use_pixel_norm, use_instance_norm, use_styles, activation_layer)
        self.conv1 = MyConv2d(out_channels, out_channels, kernel_size=3, gain=gain, use_wscale=use_wscale)
        self.epi2 = LayerEpilogue(out_channels, dlatent_size, use_wscale, use_noise, use_pixel_norm, use_instance_norm, use_styles, activation_layer)
            
    def forward(self, x, dlatents_in_range):
        x = self.conv0_up(x)
        x = self.epi1(x, dlatents_in_range[:, 0])
        x = self.conv1(x)
        x = self.epi2(x, dlatents_in_range[:, 1])
        return x

class G_synthesis(nn.Module):
    def __init__(self,
        dlatent_size        = 512,          # Disentangled latent (W) dimensionality.
        num_channels        = 3,            # Number of output color channels.
        resolution          = 1024,         # Output resolution.
        fmap_base           = 8192,         # Overall multiplier for the number of feature maps.
        fmap_decay          = 1.0,          # log2 feature map reduction when doubling the resolution.
        fmap_max            = 512,          # Maximum number of feature maps in any layer.
        use_styles          = True,         # Enable style inputs?
        const_input_layer   = True,         # First layer is a learned constant?
        use_noise           = True,         # Enable noise inputs?
        randomize_noise     = True,         # True = randomize noise inputs every time (non-deterministic), False = read noise inputs from variables.
        nonlinearity        = 'lrelu',      # Activation function: 'relu', 'lrelu'
        use_wscale          = True,         # Enable equalized learning rate?
        use_pixel_norm      = False,        # Enable pixelwise feature vector normalization?
        use_instance_norm   = True,         # Enable instance normalization?
        dtype               = torch.float32,  # Data type to use for activations and outputs.
        blur_filter         = [1,2,1],      # Low-pass filter to apply when resampling activations. None = no filtering.
        ):
        
        super().__init__()
        def nf(stage):
            return min(int(fmap_base / (2.0 ** (stage * fmap_decay))), fmap_max)
        self.dlatent_size = dlatent_size
        resolution_log2 = int(np.log2(resolution))
        assert resolution == 2**resolution_log2 and resolution >= 4

        act, gain = {'relu': (torch.relu, np.sqrt(2)),
                     'lrelu': (nn.LeakyReLU(negative_slope=0.2), np.sqrt(2))}[nonlinearity]
        num_layers = resolution_log2 * 2 - 2
        num_styles = num_layers if use_styles else 1
        torgbs = []
        blocks = []
        for res in range(2, resolution_log2 + 1):
            channels = nf(res-1)
            name = '{s}x{s}'.format(s=2**res)
            if res == 2:
                blocks.append((name,
                               InputBlock(channels, dlatent_size, const_input_layer, gain, use_wscale,
                                      use_noise, use_pixel_norm, use_instance_norm, use_styles, act)))
                
            else:
                blocks.append((name,
                               GSynthesisBlock(last_channels, channels, blur_filter, dlatent_size, gain, use_wscale, use_noise, use_pixel_norm, use_instance_norm, use_styles, act)))
            last_channels = channels
        self.torgb = MyConv2d(channels, num_channels, 1, gain=1, use_wscale=use_wscale)
        self.blocks = nn.ModuleDict(OrderedDict(blocks))
        
    def forward(self, dlatents_in):
        # Input: Disentangled latents (W) [minibatch, num_layers, dlatent_size].
        # lod_in = tf.cast(tf.get_variable('lod', initializer=np.float32(0), trainable=False), dtype)
        batch_size = dlatents_in.size(0)       
        for i, m in enumerate(self.blocks.values()):
            if i == 0:
                x = m(dlatents_in[:, 2*i:2*i+2])
            else:
                x = m(x, dlatents_in[:, 2*i:2*i+2])
        rgb = self.torgb(x)
        return rgb

Подгрузим веса

In [ ]:
# https://github.com/lernapparat/lernapparat/releases/download/v2019-02-01/karras2019stylegan-ffhq-1024x1024.for_g_all.pt
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L13/weights_stylegan.pt 
clear_output()

Определим модель и загрузим в нее обученные веса

In [ ]:
model_stylegan = nn.Sequential(OrderedDict([
    ('g_mapping', G_mapping()),
    ('g_synthesis', G_synthesis())    
]))

model_stylegan.load_state_dict(torch.load('/content/weights_stylegan.pt'))

Посмотрим на картинки

In [ ]:
torch.manual_seed(42)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model_stylegan.eval()
model_stylegan.to(device)

nb_rows = 2
nb_cols = 5
nb_samples = nb_rows * nb_cols
latents = torch.randn(nb_samples, 512, device=device)
with torch.no_grad():
    imgs = model_stylegan(latents)
    imgs = (imgs.clamp(-1, 1) + 1) / 2.0 # normalization to 0..1 range
imgs = imgs.cpu()

imgs = torchvision.utils.make_grid(imgs, nrow=nb_cols)

plt.figure(figsize=(15, 6))
plt.imshow(imgs.permute(1, 2, 0).detach().numpy())
plt.axis('off')
plt.show()

А теперь посмотрим на плавные переходы одной картинки в другую с помощью интерполяции латентного пространства

Собственно посчитаем и сохраним картинки

In [ ]:
!mkdir frames

In [ ]:
!rm -rf frames/*

from tqdm.notebook import tqdm

nb_latents = 3
nb_interp = 50

fixed_latents = [torch.randn(1, 512, device=device) for _ in range(nb_latents)]
latents = []
for i in range(len(fixed_latents) - 1):
    a = fixed_latents[i]
    b = fixed_latents[i + 1]
    latents.append(a + (b - a) * torch.linspace(0, 1, nb_interp, device=device).unsqueeze(1))
# latents.append(fixed_latents[-1])
latents = torch.cat(latents, dim=0)

with torch.no_grad():
    for n, latent in tqdm(enumerate(latents), total=len(latents)):
        latent = latent.to(device)
        img = model_stylegan(latent.unsqueeze(0))
        img = img.clamp_(-1, 1).add_(1).div_(2.0)        
        img = img.detach().squeeze(0).cpu().permute(1, 2, 0).numpy()
        plt.axis('off')
        plt.imshow(img)
        plt.savefig('/content/frames/frame_%03d.png' % n)
        plt.close()

И сделаем из них гифку

In [ ]:
import imageio
from IPython.display import Image

imgs = []
path = '/content/frames/'
for filename in np.sort(os.listdir(path)):
    imgs.append(imageio.imread(path + filename))
imageio.mimsave('StyleGAN.gif', imgs, fps=10)

Image(open('StyleGAN.gif', 'rb').read())

### StyleGAN 2

Естественно, команда авторов на этом не остановилась и спустя год предложила **StyleGAN2** - генеративную состязательную сеть, построенную на базе **StyleGAN** с рядом усовершенствований:
- адаптивная нормализация (**AdaIN**) была переработана и заменена на технику нормализации, называемую демодуляцией весов (**weight demodulation**). Теперь вектор стиля модифицирует **не значения признаков**, а **веса свертки**. **AdlIN** приводила к характерным **артефактам** в виде пятен на изображении (можно увидеть на правой щеке одного из изображений, в гифке выше)

- введена улучшенная **схема обучения** по прогрессивно растущей схеме, которая достигает той же цели - обучение начинается с фокусировки на изображениях **низкого разрешения**, а затем постепенно смещает фокус на все более **высокие разрешения** - **без изменения топологии** сети во время обучения. 
- предлагаются новые типы регуляризации, такие как ленивая регуляризация (**lazy regularization**, регуляризация [R1](https://paperswithcode.com/method/r1-regularization) для экономии вычислительных ресурсов применяется не всегда, а на каждом 16-ом батче) и регуляризация по длине пути (**path length regularization**, сеть штрафуют, если небольшое изменение вектора латентного пространство слишком сильно или слишком слабо меняет итоговое изображение).

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/style_gan_detailed_scheme.png" width="800">
<center><em>Сравнение архитектуры StyleGAN и StyleGAN2 (<a href="https://arxiv.org/pdf/1912.04958.pdf"> Karras et al., 2019</a>)</p> </em></center>

Часть **А** - это та же архитектура **StyleGAN**.

В части **В** показан детальный вид архитектуры **StyleGAN**. 

В части **C** они заменили **AdaIN** на **Modulation** (или масштабирование признаков) и **Normalization**. Кроме того, в части **C** они перенесли добавление шума и смещения за пределы блока. 

Наконец, в части **D** вы можете увидеть, что **веса** корректируются с помощью стиля, а **нормализация** заменена операцией "демодуляции" (**demodulation**), комбинированные операции называются "Демодуляция весов" (**weight demodulation**).

In [ ]:
%cd /content/
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3
clear_output()

In [ ]:
%cd /content/stylegan2-ada-pytorch
!python generate.py --outdir=out --trunc=1 --seeds=85,265,297,849 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl
clear_output()

In [ ]:
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
clear_output()

In [ ]:
with open('ffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].to(device)  # torch.nn.Module

In [ ]:
z = torch.randn([1, G.z_dim]).to(device)    # latent codes
c = None                                # class labels (not used in this example)
img = G(z, c)                           # NCHW, float32, dynamic range [-1, +1]

img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu()[0]

plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
!mkdir frames

In [ ]:
!rm -rf frames/*

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

nb_latents = 3
nb_interp = 50

fixed_latents = [torch.randn([1, G.z_dim], device=device) for _ in range(nb_latents)]
latents = []
for i in range(len(fixed_latents) - 1):
    a = fixed_latents[i]
    b = fixed_latents[i + 1]
    latents.append(a + (b - a) * torch.linspace(0, 1, nb_interp, device=device).unsqueeze(1))

latents = torch.cat(latents, dim=0)

with torch.no_grad():
    for n, latent in tqdm(enumerate(latents), total=len(latents)):
        latent = latent.to(device)
        img = G(latent.unsqueeze(0), c)
        img = img.clamp_(-1, 1).add_(1).div_(2.0)        
        img = img.detach().squeeze(0).cpu().permute(1, 2, 0).numpy()
        plt.axis('off')
        plt.imshow(img)
        plt.savefig('/content/stylegan2-ada-pytorch/frames/frame_%03d.png' % n)
        plt.close()

In [ ]:
imgs = []
path = '/content/stylegan2-ada-pytorch/frames/'
for filename in np.sort(os.listdir(path)):
    imgs.append(imageio.imread(path + filename))
imageio.mimsave('StyleGAN2.gif', imgs, fps=10)

Image(open('StyleGAN2.gif','rb').read())

### Alias-Free GAN

[Alias-Free GAN](https://nvlabs.github.io/stylegan3/) так же называют StyleGAN3.

В 2021 году авторы заметили, что, несмотря на **иерархическую природу** генерации (как мы видели у **StyleGAN** ранние слои отвечают за форму и положение головы, средние за черты лица и т.д.) генерации изображений типичными **GAN**, генерация зависит от **абсолютных координат** пикселей. Это проявляется в том, что детали оказываются приклеенными к координатам изображения, а не к поверхностям изображаемых объектов. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/compare_style_and_alias_free_gans.gif" width="800">
<center><em>Сравнение сгенерированных лиц с помощью StyleGAN2 и Anti-Alias GAN (Karras et al., 2021). Обратите внимание на то, как детали бороды (StyleGAN2) как бы "приклеиваются" к пространству.</em></center>


Это связанно с тем, что при генерации картинок используются **нелинейные** (а они должны быть нелинейными, иначе нейросеть не научиться так хорошо [аппроксимировать](https://towardsai.net/p/deep-learning/understanding-the-universal-approximation-theorem)) **функции активации** (а в случае **ReLU** еще и не гладкие). Это приводит к появлению на карте признаков (и на финальном изображении) высокачастотного шума. Идея **Alias-Free GAN** заключается в добавлении **фильтров нижних частот** **ФНЧ**, которые фильтруют такой шум. **ФНЧ** - это по сути свертка с фиксированными весами, которая убирает частые изменения признаков (рябь на картинке).

**Alias-Free GAN** соответствуют по метрике [расстояния Фреше](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%A4%D1%80%D0%B5%D1%88%D0%B5) (Fr´echet Inception distance или FID) **StyleGAN2**, но изображения, созданные с ее помощью содержат меньше заметных глазу артефактов. 


[Отличное видео summary Alias-Free GAN с привязкой к StyleGan2](https://www.youtube.com/watch?v=0zaGYLPj4Kk&ab_channel=TwoMinutePapers)

# Тонкости обучения GANов

[Статья - детальный разбор тонкностей и советов](https://beckham.nz/2021/06/28/training-gans.html)

## Частые/простые ошибки

* **Убедитесь, что сгенерированые сэмплы в том же диапазоне, что и реальные данные.** Например, реальные данные `[-1,1]`, при этом генерируются данные `[0,1]`. Это не хорошо, так как это подсказка для дискриминатора. 
* **Убедитесь, что сгенерированные сэмплы того же размера, что и реальные данные.** Например, размер картинок в MNIST `(28,28)`, а генератор выдает `(32,32)`. В таком случае нужно либо изменить архитектуру генератора, чтобы получать на выходе размер `(28,28)`, либо сделать ресайз реальных данных до `(32,32)`.
* **Старайтесь не использовать `BatchNorm`**. Проблема `BN` в том, что во время обучения его внутренняя статистика считается по минибатчу, а во время инференса она вычисляется как *moving average*, что в свою очередь может повлечь непредсказуемые результаты. Если архитектура GAN предполагает нормализацию - то лучше использовать **`InstanceNorm`**.
* **Визуализируйте свои лоссы в процессе обучения**. Для этого существует множество прекрасных библиотек (например `tensorboard`). Следить за бегущими по экрану цифрами от двух соревнующихся между собой лоссов - бессмысленно.



## Зачем давать преимущество дискриминатору

Если ваша версия **GAN** работает не так хорошо как вам хотелось бы, попробуйте дать своему **дискриминатору** преимущество, обучив его на относительно большее число итераций, чем **генератор**. Другими словами, чем лучше **дискриминатор** различает настоящие и фальшивые данные, тем лучше сигнал, который **генератор** может извлечь из него. Обратите внимание, что, эта логика не имела смысла во времена "до WGAN", поскольку слишком хорошая работа дискриминатора вредила обучению.

Например:


```
def train_on_batch(x, iter_, n_gen=5):
     Generator:
    ...
    ...
    if iter_ % n_gen == 0:
        g_loss.backward()
        opt_g.step()
        
     Discriminator:
    ...
    ...
    d_loss.backward() 
    d_loss.step()
```

Где `iter_` - текущая итерация шага градиента, а `n_gen` определяет интервал между обновлениями генератора. В данном случае, поскольку он равен 5, мы можем считать, что это означает, что дискриминатор обновляется в 5 раз чаще, чем генератор.

Естественно работает не всегда и не везде. Но попробовать стоит


## Использование оптимизатора ADAM

Можно обратить внимание, что почти во всех статьях по **GAN** используется **ADAM**. Сложно сказать, почему так получается, но он работает и работает очень хорошо. Если качество вашего **GAN** оставляет желать лучшего - скорее всего оптимизатор тут не причем. Ищите ошибку где-то еще.

Параметр `epsilon` **ADAM** по умолчанию в PyTorch равен `1e-8`, что может вызвать проблемы после длительного периода обучения, например, лоссы потери периодически взрываются или увеличиваются. Подробнее об этом на [StackOverflow](https://stackoverflow.com/questions/42327543/adam-optimizer-goes-haywire-after-200k-batches-training-loss-grows) и в комментарии на [Reddit](https://www.reddit.com/r/reinforcementlearning/comments/j9rflf/intuitive_explanation_for_adams_epsilon_parameter/).

## Top K Training

В статье [Top-k Training of GANs: Improving GAN Performance by Throwing Away Bad Samples (Sinha et al., 2020)](https://arxiv.org/abs/2002.06224) утверждается, что если просто **обнулить вклад градиента от сэмплов, которые дискриминатор считает поддельными**, генератор обучается значительно лучше, достигая нового **SOTA** (реализуется в одну строчку).

# Краткое описание примечательных моделей GAN

**GAN** моделей настолько много, что нет смысла рассказывать о всех. Так или иначе, сейчас многие новые нейросети используют принципы **GAN** для обучения. Будь-то распознавание или сегментация.

[Самые современные генеративные модели](https://paperswithcode.com/methods/category/generative-models)

[exactly how the NVIDIA GauGAN neural network works](https://sudonull.com/post/29972-Pictures-from-rough-sketches-how-exactly-the-NVIDIA-GauGAN-neural-network-works-ITSumma-Blog)

## GAN для решения задачи распознавания капчи

[Yet Another Text Captcha Solver, (Ye et all, 2018)](https://zwang4.github.io/publication/ccs18/)


В чем идея: для обучения **классифицирующей** нейронной сети требуется **много данных**. Ботов, собирающих изображения капч, легко распознать и заблокировать.

Решение: Почему бы нам не **сгенерировать** с помощью **GAN**, примеров для обучения классификатора?

Так и сделали исследователи из Великобритании и Китая, собрали всего 500 образцов от 11 сервисов капчи, используемых на 32 сайтах из топ-50 в рейтинге Alexa (рейтинг самых посещаемых сайтов от Amazon в настоящее время не поддерживается). На сбор разработчики потратили всего 2 часа. В процессе же обучения было «синтезировано» более 200 000 тысяч капч.

Удалось обойти текстовые **CAPTCHA** со 100% точностью на сайтах Megaupload, Blizzard и Authorize.NET. Кроме того, метод превзошел другие подходы при взломе капчи на Amazon, Digg, Slashdot, PayPal, Yahoo, QQ и других сайтах.

<center><img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/generate_captchas.png" width=800></center>

## Pix2Pix

[Image-to-Image Translation with Conditional Adversarial Networks (Isola et al., 2016)](https://arxiv.org/abs/1611.07004)

**Pix2Pix** - сети, переводящая пиксельные рисунки в реалистичные изображения. Она тоже использует принципы **GAN** для работы.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/pix2pix_scheme.png" width="500"></center>
<center><em>Схема работы Pix2Pix (Isola et al., 2016).</em></center>

Попробовать сеть можно на [сайте 1](https://affinelayer.com/pixsrv/) и на [сайте 2](https://affinelayer.com/pix2pix/):

Ещё примеры:


<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/pix2pix_results_examples.png" width="900"></center>
<center><em>Еще примеры работы Pix2Pix (Isola et al., 2016).</em></center>


## Семантическая генерация

Помимо **шума**, в модель можно подавать **описание** того, что мы хотим получить. Это описание может быть разным, например **текст** или **вектор заданных свойств**.

Например:
* Класс объекта
* Углы и повороты
* Заданные параметры трансформаций
* Сегментация

[Learning to Generate Chairs, Tables and Cars
with Convolutional Networks (Dosovitskiy et al., 2017)](https://arxiv.org/abs/1411.5928)

Эта статья - эксперимент на тему того, можно ли использовать **GAN** для дизайна 3D объектов, таких как стулья, столы и автомобили. При этом авторы задают параметры объекта - его класс, положение в пространстве (c какой точки мы смотрим на объект), цвет, яркость и т.д. Для сгенерированных объектов также решалась задача сегментации - это дает возможность вставить объект на произвольный фон. 

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/3d_gan_scheme.png" width="700"></center>
<center><em>Схема работы GAN, который учитывает много параметров, например угол обзора и класс (Dosovitskiy et al., 2017).</em></center>

Данная статья показала, что GAN может не только придумывать новые картинки, которые отсутствовали в train, но и понимать геометрию объекта и работать с пространственным положением. 

Из интересных результатов, данная нейронная сеть позволяет “складывать” и “вычитать” стулья. Для этого используется сложение и вычитание признаков на выходе FC-2 (см. картинку выше).

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/semantic_chairs.png" width="350"></center>
<center><em>Интерпретируемые результаты генерации при проведении арифметических операций между признаками (Dosovitskiy et al., 2017).</em></center>

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/3d_gan_results.png" width="300"></center>
<center><em>Результат генерации и интерполяции между машинками (Dosovitskiy et al., 2017).</em></center>

## Text to image

Существует задача генерации изображения по текстовому описанию. Такую задачу можно решать используя в качестве части источника случайности текстовое описание. 

Одна из возможных архитектур сетей text-to-image с использованием RNN-сети на входе:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/tex_to_image_gan_scheme.png" width="800"></center>
<center><em>Схема работы Generative Adversarial Text to Image Synthesis (Reed et al., 2016) </em></center>

[Generative Adversarial Text to Image Synthesis (Reed et al., 2016)](https://arxiv.org/pdf/1605.05396.pdf)

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/control_gan_results.png" width="500">

<em>Результаты работы Adversarial Text to Image Synthesis (Reed et al., 2016).</em></center>


### ControlGAN

[Controllable Generative Adversarial Network (Lee et al., 2017)](https://arxiv.org/abs/1708.00598)

[Код ControlGAN](https://github.com/mrlibw/ControlGAN) - Pytorch реализация  для управляемого преобразования текста в изображение. На входе используется эмбеддинг слов, который подаётся во множество генераторов и дискриминаторов.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/control_gan_scheme.jpg" width="800"></center>
<center><em>Схема работы ControlGAN (https://github.com/mrlibw/ControlGAN).</em></center>



Поэкспериментировать:

[Colab, где можно вводить собственный текст](https://colab.research.google.com/drive/1_3sCa9QvUI0OqWqx2rQvsSh3C9xfPssZ?usp=sharing)

[Open AI DALL-E интерактив](https://openai.com/blog/dall-e/)

[OpenSource версия DALL-E](https://colab.research.google.com/drive/1b8va5g852hq3p7yro7xWY3Cc-bd2CRdv)

Более современная архитектура с использованием механизма внимания:

[Text-to-Image Generation with Attention Based Recurrent Neural Networks (Zia et al., 2020)](https://arxiv.org/abs/2001.06658)

### VQGAN + CLIP

[ Пример тетради для работы с VQGAN + CLIP](https://colab.research.google.com/drive/1ZAus_gn2RhTZWzOWUpPERNC0Q8OhZRTZ)

<center>

<img src = "https://edunet.kea.su/repo/EduNet-content/L13/out/vqgan_clip_result.png" width='300'>

</center>

<center><em>Изображение сгенерировано нейросетью (запрос: self-conciousness, artstation)</em></center>

## Задача переноса стиля
    
[PapersWithCode Image Stylization](https://paperswithcode.com/task/image-stylization)

### Постановка задачи

Перенос стиля (style transfer)  — одно из наиболее креативных приложений сверточных нейронных сетей. Взяв контент с одного изображения и стиль от второго, нейронная сеть объединяет их в одно художественное произведение.


<img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/style_transfer_result.png" width="800">


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/style_transfer_input_reference_result.png" width="600">

Для целей переноса стиля [можно использовать GAN](https://towardsdatascience.com/style-transfer-with-gans-on-hd-images-88e8efcf3716) (но так делают редко):



<center><img src ="https://edunet.kea.su/repo/EduNet-content/L13/out/gan_for_style_transfer.png" width="600"></center>
<center><em>Сеть состоит из одного генератора (G) и сиамского дискриминатора (D): G принимает изображение-контент (А) на входе и выводит некое итоговое изображение G(A). Дискриминатор  принимает эти изображения и изображение-стиль (B) в качестве входных данных и выводит скрытый вектор. Siamese Discriminator преследует 2 цели: дать информацию генератору, как создавать более реалистичные изображения, и поддерживать в этих поддельных изображениях корреляцию (то есть «контент») с исходными.</em></center>


### Style Flow

[Style Flow](https://arxiv.org/pdf/2008.02401.pdf)


Управление процессом генерации с использованием (семантических) атрибутов, сохраняя при этом качество выходных данных


### Image-to-Image Translation

Мы не будем вдаваться в подробности как работают следующие модели, но мы предлагаем вам ссылки на Colab`ы где с ними можно самостоятельно поиграться

#### **GANs N' Roses**

[GANs N' Roses: Stable, Controllable, Diverse Image to Image Translation (Chong et al., 2021)](https://arxiv.org/abs/2106.06561)

[Colab](https://colab.research.google.com/github/mchong6/GANsNRoses/blob/main/inference_colab.ipynb)


<center><img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/image_to_image_result.gif"></center>
<center><em>Результаты работы GANs N' Roses (Chong et al., 2021)</em></center>


#### **CycleGAN**

[Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks (Zhu et al., 2017)](https://arxiv.org/abs/1703.10593)

[Colab](https://colab.research.google.com/github/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/CycleGAN.ipynb)

<center><img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L13/cycle_gan_horse_to_zebra.gif" width="800"/></center>
<center><em>Результаты работы CycleGAN (Zhu et al., 2017)</em></center>



<font size ="6"> Заключение

Принципы генеративно-состязательных сетей вышли далеко за пределы генерации из шума. Сейчас с помощью GAN создаются сложнейшие state-of-art сети для самых разнообразных задач. В лекции были рассмотрены самые главные модели: GAN - для генерации из шума, DCGAN - для генерации изображений, с помощью развёрток и cGAN - сети с генерацией по условию. 

Совмещая различные сети и подходы можно сконструировать множество самых разнообразных моделей.

<font size="6"> Использованная литература

<font size="5"> GAN

[Книга по генеративным сетям](https://habr.com/ru/company/piter/blog/504956/)

[Generative Adversarial Networks (Goodfellow et al., 2014)](https://arxiv.org/abs/1406.2661)

[Видео разбор оригинальной статьи GAN](https://youtu.be/eyxmSmjmNS0)

[Видео лекции Иана Гудфеллоу](https://www.youtube.com/watch?v=HGYYEUSm-0Q)

[Generative adversarial networks](https://deepgenerativemodels.github.io/notes/gan/)

[Самые современные генеративные модели](https://paperswithcode.com/methods/category/generative-models)

[exactly how the NVIDIA GauGAN neural network works](https://sudonull.com/post/29972-Pictures-from-rough-sketches-how-exactly-the-NVIDIA-GauGAN-neural-network-works-ITSumma-Blog)

<font size="5"> DCGAN

[Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks (Radford et al., 2015)](https://arxiv.org/abs/1511.06434).

[DCGAN TUTORIAL](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)

<font size="5"> Wassserstein GAN

[Wasserstein GAN (Arjovsky et. al., 2017)](https://arxiv.org/abs/1701.07875)

[Блог пост о Wasserstein GAN](https://vincentherrmann.github.io/blog/wasserstein/)

[Improved Training of Wasserstein GANs (Gulrajani et al., 2017)](https://arxiv.org/abs/1704.00028)

[Spectral Normalization for Generative Adversarial Networks (Miyato et al., 2018)](https://arxiv.org/abs/1802.05957).

<font size="5"> ProGAN -> StyleGAN -> StyleGAN2 -> Alias-Free GAN

[Progressive Growing of GANs for Improved Quality, Stability, and Variation (ProGAN) [Karras et al., 2017]](https://arxiv.org/abs/1710.10196)

[A Style-Based Generator Architecture for Generative Adversarial Networks (StyleGAN) [Karras et al., 2018]](https://arxiv.org/abs/1812.04948)

[Analyzing and Improving the Image Quality of StyleGAN (StyleGAN2) [Karras et al., 2019]](https://arxiv.org/abs/1912.04958)

[Alias-Free Generative Adversarial Networks (Alias-Free GAN) [Karras et al., 2021]](https://arxiv.org/abs/2106.12423)

<font size="5"> Тонкости обучения GAN

[Статья - детальный разбор тонкностей и советов](https://beckham.nz/2021/06/28/training-gans.html)

[Top-k Training of GANs: Improving GAN Performance by Throwing Away Bad Samples (Sinha et al., 2020)](https://arxiv.org/abs/2002.06224)

<font size="5"> GAN Zoo:

<font size="5"> BigGAN

[Large Scale GAN Training for High Fidelity Natural Image Synthesis (Brock et al., 2018)](https://arxiv.org/abs/1809.11096)

<font size="5"> StackGAN

[StackGAN: Text to Photo-realistic Image Synthesis with Stacked Generative Adversarial Networks (Zhang et al., 2016)](https://arxiv.org/abs/1612.03242)

[StackGAN++: Realistic Image Synthesis with Stacked Generative Adversarial Networks (Zhang et al., 2017)](https://arxiv.org/abs/1710.10916)

[Photo-Realistic Single Image Super-Resolution Using a Generative Adversarial Network (Ledig et al., 2016)](https://arxiv.org/abs/1609.04802)

[Let’s Read Science! “StackGAN: Text to Photo-Realistic Image Synthesis”](https://medium.com/@rangerscience/lets-read-science-stackgan-text-to-photo-realistic-image-synthesis-4562b2b14059)

[Deep Learning Generative Models for Image Synthesis and Image Translation](https://www.rulit.me/data/programs/resources/pdf/Generative-Adversarial-Networks-with-Python_RuLit_Me_610886.pdf)

[youtube [StackGAN++] Realistic Image Synthesis with Stacked Generative Adversarial Networks | AISC](https://www.youtube.com/watch?v=PXWIaLE7_NU)

[youtube Text to Photo-realistic Image Synthesis with Stacked Generative Adversarial Networks](https://www.youtube.com/watch?v=crI5K4RCZws)

<font size="5"> ControlGAN

[Controllable Generative Adversarial Network](https://arxiv.org/pdf/1708.00598.pdf)

[Controllable Text-to-Image Generation](https://arxiv.org/pdf/1909.07083.pdf)

[Image Generation and Recognition (Emotions)](https://arxiv.org/pdf/1910.05774.pdf)

[Natural Language & Text-to-Image 2019](https://meta-guide.com/data/data-processing/text-to-image-systems/natural-language-text-to-image-2019)

<font size="5"> AC-GAN

[How to Develop an Auxiliary Classifier GAN (AC-GAN) From Scratch with Keras](https://machinelearningmastery.com/how-to-develop-an-auxiliary-classifier-gan-ac-gan-from-scratch-with-keras/)

[Understanding ACGANs with code[PyTorch]](https://towardsdatascience.com/understanding-acgans-with-code-pytorch-2de35e05d3e4)

[An Auxiliary Classifier Generative Adversarial Framework for Relation Extraction](https://arxiv.org/pdf/1909.05370.pdf)

[A Multi-Class Hinge Loss for Conditional GANs](https://openaccess.thecvf.com/content/WACV2021/papers/Kavalerov_A_Multi-Class_Hinge_Loss_for_Conditional_GANs_WACV_2021_paper.pdf)

<font size="5"> Domain Transfer Network

[Unsupervised Cross-Domain Image Generation (Taigma et al., 2016)](https://arxiv.org/abs/1611.02200)

<font size="5"> Pix2Pix

[Image-to-Image Translation with Conditional Adversarial Networks (Isola et al., 2016)](https://arxiv.org/abs/1611.07004)

<font size="5"> Семантическая генерация

[Learning to Generate Chairs, Tables and Cars
with Convolutional Networks (Dosovitskiy et al., 2017)](https://arxiv.org/abs/1411.5928)

<font size="5"> Text to image

[Text-to-Image Generation with Attention Based Recurrent Neural Networks (Zia et al., 2020)](https://arxiv.org/abs/2001.06658)

<font size="5">Image-to-Image

[GANs N' Roses: Stable, Controllable, Diverse Image to Image Translation (Chong et al., 2021)](https://arxiv.org/abs/2106.06561)

[Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks (Zhu et al., 2017)](https://arxiv.org/abs/1703.10593)

<font size="5"> Ссылки

[GitHub MNIST CelebA cGAN cDCGAN](https://github.com/znxlwm/pytorch-MNIST-CelebA-cGAN-cDCGAN)

[GitHub Text-to-Photo realistic Image Synthesis with Stacked Generative Adversarial Networks](https://github.com/zeusm9/Text-to-Photo-realistic-Image-Synthesis-with-Stacked-Generative-Adversarial-Networks)

[GitHub ControlGAN](https://github.com/mrlibw/ControlGAN)

[GitHub ControlGAN-Tensorflow](https://github.com/taki0112/ControlGAN-Tensorflow)

[GitHub Keras-ACGan](https://github.com/lukedeo/keras-acgan)

[Множество примеров различных генераторов](https://thisxdoesnotexist.com)
